In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split

In [ ]:
FAST_RUN = False
IMAGE_WIDTH=450
IMAGE_HEIGHT=450
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
batch_size = 64
IMAGE_CHANNELS=3
input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)

In [ ]:
import zipfile


zip_files = ['test', 'train']
# Will unzip the files so that you can see them..
for zip_file in zip_files:
    with zipfile.ZipFile("../input/dogs-vs-cats-redux-kernels-edition/{}.zip".format(zip_file),"r") as z:
        z.extractall(".")
        print("{} unzipped".format(zip_file))

In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread

folder = '/kaggle/working/train/'

for i in range(9):
    pyplot.subplot(330+1+i)
    file = folder + 'cat.' + str(i) + '.jpg'
    image = imread(file)
    pyplot.imshow(image)
    
pyplot.show()

In [ ]:
folder = '/kaggle/working/train/'

for i in range(9):
    pyplot.subplot(330+1+i)
    file = folder + 'dog.' + str(i) + '.jpg'
    image = imread(file)
    pyplot.imshow(image)
    
pyplot.show()

In [ ]:
filenames = os.listdir('/kaggle/working/train')
categories = []

for file in filenames:
    if ( file.split('.')[0] == 'dog'):
        categories.append('dog')
    else:
        categories.append('cat')
        
df=pd.DataFrame({'filename' : filenames, 'categories' : categories})
df.head()

In [ ]:
df.tail()

In [ ]:
df['categories'].value_counts()

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=1)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
train_df['categories'].value_counts()

In [ ]:
train_df.head(20)

In [ ]:
validate_df.head(20)

In [ ]:
validate_df['categories'].value_counts()

In [ ]:
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "/kaggle/working/train/", 
    x_col='filename',
    y_col='categories',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/kaggle/working/train/", 
    x_col='filename',
    y_col='categories',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
example_df = train_df.sample(n=1).reset_index(drop=True)
example_generator = train_datagen.flow_from_dataframe(
    example_df, 
    "/kaggle/working/train/", 
    x_col='filename',
    y_col='categories',
    target_size=IMAGE_SIZE,
    class_mode='categorical'
)
pyplot.figure(figsize=(12, 12))
for i in range(0, 15):
    pyplot.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        pyplot.imshow(image)
        break
pyplot.tight_layout()
pyplot.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

In [ ]:
model = Sequential()
batch_size

In [ ]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax')) 

In [ ]:
model=Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax')) 

In [ ]:
from keras.optimizers import SGD
opt= SGD(lr=0.001, momentum=0.9)

model.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

earlystop = EarlyStopping(patience=20)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

In [ ]:
'''Hi Im Krisha hhnf nnnnnmdfnmnm hklkjgjjfhhfhmnhhhhjhjhhjyjdtyjdt'''

In [ ]:
#model.load_weights('../input/model8/model (8).h5')

In [ ]:
epochs = 30 
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=5000//batch_size,
    steps_per_epoch=20000//batch_size,
    callbacks=callbacks
)

In [ ]:
model.save_weights("model.h5")
file_path = "best_model.hdf5"
 
#check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1, mode = "min")


In [ ]:
test_filenames = os.listdir("/kaggle/working/test")
test_df = pd.DataFrame({
    'filename': test_filenames
    
})
nb_samples = test_df.shape[0]

In [ ]:
nb_samples

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "/kaggle/working/test", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
#test_generator = test_gen.flow(test_df['filename'], batch_size=batch_size)
prediction_probabilities = model.predict_generator(test_generator, verbose=1)

In [ ]:
test_df['categories'] = np.argmax(prediction_probabilities, axis=-1)

In [ ]:
test_df['categories'].value_counts()

In [ ]:
test_df['sorted'] = test_df.filename.str.extract("(.*).jpg").astype(int)
#df.sort_values('sorted',inplace=True, ascending=False)
test_df = test_df.sort_values(['sorted'])
test_df.head()

In [ ]:
test_df.drop('filename', axis =1)

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['sorted']
submission_df['label'] = submission_df['categories']
submission_df.drop(['sorted', 'categories', 'filename'], axis=1, inplace=True)
submission_df.reset_index()
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df.head()

In [ ]:
example_df = test_df.sample(n=1).reset_index(drop=True)
example_generator = test_gen.flow_from_dataframe(
    example_df, 
    "/kaggle/working/test", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)
pyplot.figure(figsize=(12, 12))
pyplot.subplot(5, 3, i+1)
for X_batch in example_generator:
    image = X_batch[0]
    pyplot.imshow(image)
    break
pyplot.tight_layout()
pyplot.show()

In [ ]:
prediction = model.predict_generator(example_generator, verbose=1)

In [ ]:
prediction